In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
pokemon_df = pd.read_csv('pokemon.csv')
pokemon_df = pokemon_df.set_index('#')
combats_df = pd.read_csv('combats.csv')
pokemon_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 800 entries, 1 to 800
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Name        799 non-null    object
 1   Type 1      800 non-null    object
 2   Type 2      414 non-null    object
 3   HP          800 non-null    int64 
 4   Attack      800 non-null    int64 
 5   Defense     800 non-null    int64 
 6   Sp. Atk     800 non-null    int64 
 7   Sp. Def     800 non-null    int64 
 8   Speed       800 non-null    int64 
 9   Generation  800 non-null    int64 
 10  Legendary   800 non-null    bool  
dtypes: bool(1), int64(7), object(3)
memory usage: 69.5+ KB


In [3]:
pokemon_df['Type 2'].fillna('empty',inplace=True)
pokemon_df['Type 2'].value_counts()

empty       386
Flying       97
Ground       35
Poison       34
Psychic      33
Fighting     26
Grass        25
Fairy        23
Steel        22
Dark         20
Dragon       18
Ice          14
Rock         14
Water        14
Ghost        14
Fire         12
Electric      6
Normal        4
Bug           3
Name: Type 2, dtype: int64

In [4]:
pokemon_df['Type 1'] = pokemon_df['Type 1'].astype('category')
pokemon_df['Type 2'] = pokemon_df['Type 2'].astype('category')
pokemon_df['Legendary'] = pokemon_df['Legendary'].astype('int64')
pokemon_df.dtypes

Name            object
Type 1        category
Type 2        category
HP               int64
Attack           int64
Defense          int64
Sp. Atk          int64
Sp. Def          int64
Speed            int64
Generation       int64
Legendary        int64
dtype: object

In [5]:
df_type1_one_hot = pd.get_dummies(pokemon_df['Type 1'])
df_type2_one_hot = pd.get_dummies(pokemon_df['Type 2'])
combine = df_type1_one_hot.add(df_type2_one_hot,fill_value=0).astype('int64')
pd.options.display.max_columns = 30
pokemon_df = pokemon_df.join(combine)
pokemon_df.head

<bound method NDFrame.head of                Name   Type 1  Type 2  HP  Attack  Defense  Sp. Atk  Sp. Def  \
#                                                                             
1         Bulbasaur    Grass  Poison  45      49       49       65       65   
2           Ivysaur    Grass  Poison  60      62       63       80       80   
3          Venusaur    Grass  Poison  80      82       83      100      100   
4     Mega Venusaur    Grass  Poison  80     100      123      122      120   
5        Charmander     Fire   empty  39      52       43       60       50   
..              ...      ...     ...  ..     ...      ...      ...      ...   
796         Diancie     Rock   Fairy  50     100      150      100      150   
797    Mega Diancie     Rock   Fairy  50     160      110      160      110   
798  Hoopa Confined  Psychic   Ghost  80     110       60      150      130   
799   Hoopa Unbound  Psychic    Dark  80     160       60      170      130   
800       Volcanion   

In [6]:
pokemon_df['Type 1'] = pokemon_df['Type 1'].cat.codes
pokemon_df['Type 2'] = pokemon_df['Type 2'].cat.codes
pokemon_df.head

<bound method NDFrame.head of                Name  Type 1  Type 2  HP  Attack  Defense  Sp. Atk  Sp. Def  \
#                                                                            
1         Bulbasaur       9      13  45      49       49       65       65   
2           Ivysaur       9      13  60      62       63       80       80   
3          Venusaur       9      13  80      82       83      100      100   
4     Mega Venusaur       9      13  80     100      123      122      120   
5        Charmander       6      18  39      52       43       60       50   
..              ...     ...     ...  ..     ...      ...      ...      ...   
796         Diancie      15       4  50     100      150      100      150   
797    Mega Diancie      15       4  50     160      110      160      110   
798  Hoopa Confined      14       8  80     110       60      150      130   
799   Hoopa Unbound      14       1  80     160       60      170      130   
800       Volcanion       6      1

In [7]:
pokemon_df.drop('Name', axis='columns',inplace=True)
pokemon_df.head()

,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Bug,Dark,Dragon,Electric,Fairy,Fighting,Fire,Flying,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water,empty
#,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,9,13,45,49,49,65,65,45,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
2,9,13,60,62,63,80,80,60,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
3,9,13,80,82,83,100,100,80,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
4,9,13,80,100,123,122,120,80,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
5,6,18,39,52,43,60,50,65,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1


In [8]:
combats_df['Winner'] = combats_df.apply(lambda x:0
                                        if x.Winner == x.First_pokemon else 1,
                                        axis='columns')
combats_df.head()

,First_pokemon,Second_pokemon,Winner
0,266,298,1
1,702,701,1
2,191,668,1
3,237,683,1
4,151,231,0


In [9]:
data_num = combats_df.shape[0]
indexes = np.random.permutation(data_num)
train_indexes = indexes[:int(data_num*0.6)]
val_indexes = indexes[int(data_num*0.6):int(data_num*0.8)]
test_indexes = indexes[int(data_num*0.8):]
train_data = combats_df.loc[train_indexes]
val_data = combats_df.loc[val_indexes]
test_data = combats_df.loc[test_indexes]


In [10]:
pokemon_df['Type 1'] = pokemon_df['Type 1']/19
pokemon_df['Type 2'] = pokemon_df['Type 2']/19

mean  = pokemon_df.loc[:,"HP":'Generation'].mean()
std = pokemon_df.loc[:,"HP":'Generation'].std()
pokemon_df.loc[:,"HP":'Generation'] = (pokemon_df.loc[:,"HP":'Generation']-mean) / std

pokemon_df.head()

,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Bug,Dark,Dragon,Electric,Fairy,Fighting,Fire,Flying,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water,empty
#,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.473684,0.684211,-0.950032,-0.924328,-0.796655,-0.238981,-0.248033,-0.801002,-1.398762,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
2,0.473684,0.684211,-0.362595,-0.523803,-0.347700,0.219422,0.290974,-0.284837,-1.398762,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
3,0.473684,0.684211,0.420654,0.092390,0.293665,0.830626,1.009651,0.403383,-1.398762,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
4,0.473684,0.684211,0.420654,0.646964,1.576395,1.502951,1.728328,0.403383,-1.398762,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
5,0.315789,0.947368,-1.185007,-0.831899,-0.989065,-0.391782,-0.787041,-0.112782,-1.398762,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
